In [2]:
### add NLP dependences
import sys
sys.path.append('drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/')
from utils import process_tweet, tools
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as MaxEnt

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Load data and Preprocess

In [4]:
### Load data
data = pd.read_csv("drive/MyDrive/Code/DETOXIS/Data/train.csv")

In [5]:
### check data/DataFrame
data.head()

,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,target_person,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,Pensó: Zumo para restar.,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,Como les gusta el afeitado en seco a esta gente.,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,"asi me gusta, que se maten entre ellos y en al...",0,0,0,0,0,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,"Loss mas valientes, los que mejor cortan nuest...",0,0,0,0,1,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,Costumbres...,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1


In [6]:
# Classes task1 and task 2
print('proportion classes task 1:\n',data['toxicity'].value_counts(),'\n')
print('proportion classes task 2:\n', data['toxicity_level'].value_counts())

proportion classes task 1:
 0    2316
1    1147
Name: toxicity, dtype: int64 

proportion classes task 2:
 0    2317
1     808
2     269
3      69
Name: toxicity_level, dtype: int64


In [7]:
### create DataFrame with relevante information
data_train = data.loc[:,['comment','toxicity','toxicity_level']]

### check data/DataFrame
data_train.head()

,comment,toxicity,toxicity_level
0,Pensó: Zumo para restar.,1,1
1,Como les gusta el afeitado en seco a esta gente.,1,1
2,"asi me gusta, que se maten entre ellos y en al...",1,2
3,"Loss mas valientes, los que mejor cortan nuest...",1,1
4,Costumbres...,1,1


#Global dependeces

In [8]:
### List of vectorization
vec_l = ['tfidf','occur']

### Feature extraction
ngrams = [(1,1), (1,2), (1,3)]

### Criate dictinaril to save import variables
pipeline_dict = { k:{ ngr: {'vect':0 , 'X':0, 'skf':0 } for ngr in ngrams } for k in vec_l }

In [9]:
### NB Criate classifier
liblinear = MaxEnt(solver='liblinear',max_iter=5000)
newton = MaxEnt(solver='newton-cg',max_iter=5000)
sag = MaxEnt(solver='sag',max_iter=5000)
saga = MaxEnt(solver='saga',max_iter=5000)
lbfgs = MaxEnt(solver='lbfgs',max_iter=5000)

### NB classifiers
clfs = {'liblinear_ME':liblinear, 'newton_ME':newton, 'sag_ME':sag, 'saga_ME':saga, 'lbfgs_ME':lbfgs}
# clfs = {'liblinear_ME':liblinear}

# Task 1

In [ ]:
### Label task1
y_task1 = data.toxicity

### Evalute matrics
metrics_task1 = ['accuracy', 'f1', 'recall', 'precision']

### Criate dictinaril results
results_dict_task1 = { k:{ vec:{ ng:{m:[] for m in metrics_task1} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

### Criate dictinaril results
mean_results_task1 = { k:{ vec:{ ng:{m:0 for m in metrics_task1} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

In [ ]:
### feed all information to the class tools to train model for task1
task1 = tools(data=data_train, 
              vectorization=vec_l, 
              ngrams=ngrams, 
              pipeline=pipeline_dict, 
              labels=y_task1, 
              models=clfs, 
              metrics=metrics_task1, 
              results=results_dict_task1, 
              mean_results=mean_results_task1)

In [ ]:
# run fuction preprocess_data()
data_train = task1.preprocess_data()

### check data/DataFrame
data_train.tail()

,comment,toxicity,toxicity_level,clean_comments_list,clean_comments_string
3458,Ya decía yo que veía menos moros,1,1,"[dec, vei, men, mor]",dec vei men mor
3459,+1. Como lo sabes...,0,0,"[numtag, sab, ...]",numtag sab ...
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1,"[segu, cobr, paguit, marruec, expoli, contribu...",segu cobr paguit marruec expoli contribuyent e...
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1,"[pobr, arriesg, pater, huy, guerr, ..., ohhh, ...",pobr arriesg pater huy guerr ... ohhh whait ...
3462,"Yo me quiero escapar también, dan paguita al l...",1,1,"[quier, escap, dan, paguit, lleg]",quier escap dan paguit lleg


In [ ]:
#### Execute function 'feature_extraction' from tools class to Vectorize and extract features from twittes
task1.feature_extraction()

In [ ]:
### Apply function "train_models" from tools class to train the and test the models (10-StratifiedKfold)
task1.train_models()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
### Apply function "average_results" from tools class to average the results whithin the 10 foldes
task1.average_results(average_dict=mean_results_task1)

In [ ]:
### Apply function "reate_Data_Frame" 
df_results_task1 = task1.create_Data_Frame()

In [ ]:
### save results to a CSV file
file_name = 'ME_results_task1'
df_results_task1.to_csv(r'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/Results/' + file_name + '.csv')

In [ ]:
### Results task 1
df_results_task1

accuracy        f1    recall  precision
liblinear_ME tfidf (1, 1)  0.711188  0.303147  0.202090   0.688248
                   (1, 2)  0.697621  0.154633  0.094851   0.878563
                   (1, 3)  0.689251  0.103558  0.063501   0.650000
             occur (1, 1)  0.699355  0.465244  0.399306   0.564882
                   (1, 2)  0.711770  0.435252  0.343394   0.605985
                   (1, 3)  0.712635  0.410124  0.312815   0.618766
newton_ME    tfidf (1, 1)  0.711477  0.304299  0.202960   0.689001
                   (1, 2)  0.697910  0.154714  0.094851   0.892849
                   (1, 3)  0.689251  0.103558  0.063501   0.650000
             occur (1, 1)  0.699066  0.464548  0.398436   0.564657
                   (1, 2)  0.710614  0.433082  0.341640   0.602960
                   (1, 3)  0.713213  0.410630  0.312815   0.621218
sag_ME       tfidf (1, 1)  0.711477  0.304299  0.202960   0.689001
                   (1, 2)  0.697910  0.154714  0.094851   0.892849
                   (1, 3)  0.689251  0.103558  0.063501   0.650000
             occur (1, 1)  0.700222  0.467890  0.401915   0.567011
                   (1, 2)  0.714080  0.442737  0.351243   0.610962
                   (1, 3)  0.715519  0.416768  0.318886   0.624821
saga_ME      tfidf (1, 1)  0.711188  0.303147  0.202090   0.688248
                   (1, 2)  0.697909  0.156154  0.095721   0.879952
                   (1, 3)  0.689251  0.103558  0.063501   0.650000
             occur (1, 1)  0.698492  0.465197  0.400175   0.562916
                   (1, 2)  0.713500  0.443215  0.352121   0.610161
                   (1, 3)  0.718116  0.424574  0.324157   0.633685
lbfgs_ME     tfidf (1, 1)  0.711477  0.304299  0.202960   0.689001
                   (1, 2)  0.697910  0.154714  0.094851   0.892849
                   (1, 3)  0.689251  0.103558  0.063501   0.650000
             occur (1, 1)  0.699066  0.464548  0.398436   0.564657
                   (1, 2)  0.710614  0.433082  0.341640   0.602960
                   (1, 3)  0.713213  0.410630  0.312815   0.621218

In [ ]:
#### Generate table for Talex
label_task1_table = 'ME_models_task1'
print(df_results_task1.to_latex(multicolumn=True, multirow=False, label=label_task1_table))

\begin{table}
\centering
\label{ME_models_task1}
\begin{tabular}{lllrrrr}
\toprule
         &       &        &  accuracy &        f1 &    recall &  precision \\
\midrule
liblinear\_ME & tfidf & (1, 1) &  0.711188 &  0.303147 &  0.202090 &   0.688248 \\
         &       & (1, 2) &  0.697621 &  0.154633 &  0.094851 &   0.878563 \\
         &       & (1, 3) &  0.689251 &  0.103558 &  0.063501 &   0.650000 \\
         & occur & (1, 1) &  0.699355 &  0.465244 &  0.399306 &   0.564882 \\
         &       & (1, 2) &  0.711770 &  0.435252 &  0.343394 &   0.605985 \\
         &       & (1, 3) &  0.712635 &  0.410124 &  0.312815 &   0.618766 \\
newton\_ME & tfidf & (1, 1) &  0.711477 &  0.304299 &  0.202960 &   0.689001 \\
         &       & (1, 2) &  0.697910 &  0.154714 &  0.094851 &   0.892849 \\
         &       & (1, 3) &  0.689251 &  0.103558 &  0.063501 &   0.650000 \\
         & occur & (1, 1) &  0.699066 &  0.464548 &  0.398436 &   0.564657 \\
         &       & (1, 2) &  0.710614 &  0.

#Task 2

In [10]:
### Label task2
y_task2 = data.toxicity_level

### Evalute matrics
metrics_task2 = ['accuracy', 'f1_macro', 'f1_weighted', 'recall', 'precision', 'cem']

### Criate dictinaril results
results_dict_task2 = { k:{ vec:{ ng:{m:[] for m in metrics_task2} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

### Criate dictinaril results
mean_results_task2 = { k:{ vec:{ ng:{m:0 for m in metrics_task2} for ng in ngrams} for vec in vec_l } for k,v in clfs.items() }

In [11]:
### feed all information to the class tools to train model for task2
task2 = tools(data=data_train, 
              vectorization=vec_l, 
              ngrams=ngrams, 
              pipeline=pipeline_dict, 
              labels=y_task2, 
              models=clfs, 
              metrics=metrics_task2, 
              results=results_dict_task2, 
              mean_results=mean_results_task2)

In [12]:
# run fuction preprocess_data()
data_train = task2.preprocess_data()

### check data/DataFrame
data_train.tail()

,comment,toxicity,toxicity_level,clean_comments_list,clean_comments_string
3458,Ya decía yo que veía menos moros,1,1,"[dec, vei, men, mor]",dec vei men mor
3459,+1. Como lo sabes...,0,0,"[numtag, sab, ...]",numtag sab ...
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1,"[segu, cobr, paguit, marruec, expoli, contribu...",segu cobr paguit marruec expoli contribuyent e...
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1,"[pobr, arriesg, pater, huy, guerr, ..., ohhh, ...",pobr arriesg pater huy guerr ... ohhh whait ...
3462,"Yo me quiero escapar también, dan paguita al l...",1,1,"[quier, escap, dan, paguit, lleg]",quier escap dan paguit lleg


In [13]:
#### Execute function 'feature_extraction' from tools class to Vectorize and extract features from twittes
task2.feature_extraction()

In [14]:
### Apply function "train_models" from tools class to train the and test the models (10-StratifiedKfold)
task2.train_models(average_recall='macro', average_precision='macro',  multiclass='True')

In [15]:
### Apply function "average_results" from tools class to average the results whithin the 10 foldes
task2.average_results(average_dict=mean_results_task2)

In [16]:
### Apply function "reate_Data_Frame" 
df_results_task2 = task2.create_Data_Frame()

In [17]:
### save results to a CSV file
file_name = 'ME_results_task2'
df_results_task2.to_csv(r'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Statistical_Models/Results/' + file_name + '.csv')

In [18]:
### Results task 2
df_results_task2

accuracy  f1_macro  ...  precision       cem
liblinear_ME tfidf (1, 1)  0.682621  0.236300  ...   0.279836  0.707041
                   (1, 2)  0.680890  0.223270  ...   0.269334  0.697224
                   (1, 3)  0.679736  0.221426  ...   0.264943  0.692280
             occur (1, 1)  0.652593  0.323639  ...   0.447263  0.683130
                   (1, 2)  0.672225  0.307046  ...   0.453852  0.701811
                   (1, 3)  0.678005  0.294448  ...   0.441820  0.707983
newton_ME    tfidf (1, 1)  0.682614  0.250871  ...   0.319947  0.706660
                   (1, 2)  0.684636  0.230160  ...   0.293446  0.704073
                   (1, 3)  0.682905  0.226693  ...   0.264891  0.697674
             occur (1, 1)  0.646528  0.358689  ...   0.494217  0.682728
                   (1, 2)  0.668181  0.317650  ...   0.450442  0.698359
                   (1, 3)  0.673957  0.299654  ...   0.430341  0.702235
sag_ME       tfidf (1, 1)  0.682614  0.250871  ...   0.319947  0.706660
                   (1, 2)  0.684636  0.230160  ...   0.293446  0.704073
                   (1, 3)  0.682905  0.226693  ...   0.264891  0.697674
             occur (1, 1)  0.645950  0.339336  ...   0.438580  0.682436
                   (1, 2)  0.669048  0.310113  ...   0.430595  0.699661
                   (1, 3)  0.674247  0.291875  ...   0.397618  0.703889
saga_ME      tfidf (1, 1)  0.682614  0.250871  ...   0.319947  0.706660
                   (1, 2)  0.684636  0.230160  ...   0.293446  0.704073
                   (1, 3)  0.682905  0.226693  ...   0.264891  0.697674
             occur (1, 1)  0.645949  0.338036  ...   0.438664  0.682463
                   (1, 2)  0.669914  0.314460  ...   0.452604  0.701504
                   (1, 3)  0.676269  0.290167  ...   0.402784  0.705277
lbfgs_ME     tfidf (1, 1)  0.682614  0.250871  ...   0.319947  0.706660
                   (1, 2)  0.684636  0.230160  ...   0.293446  0.704073
                   (1, 3)  0.682905  0.226693  ...   0.264891  0.697674
             occur (1, 1)  0.646528  0.358689  ...   0.494217  0.682728
                   (1, 2)  0.668181  0.317650  ...   0.450442  0.698359
                   (1, 3)  0.673957  0.299654  ...   0.430341  0.702235

[30 rows x 6 columns]

In [19]:
#### Generate table for Talex
label_task2_table = 'NB_models_task2'
print(df_results_task2.to_latex(multicolumn=True, multirow=False, label=label_task2_table))

\begin{table}
\centering
\label{NB_models_task2}
\begin{tabular}{lllrrrrrr}
\toprule
         &       &        &  accuracy &  f1\_macro &  f1\_weighted &    recall &  precision &       cem \\
\midrule
liblinear\_ME & tfidf & (1, 1) &  0.682621 &  0.236300 &     0.575278 &  0.269144 &   0.279836 &  0.707041 \\
         &       & (1, 2) &  0.680890 &  0.223270 &     0.560956 &  0.262855 &   0.269334 &  0.697224 \\
         &       & (1, 3) &  0.679736 &  0.221426 &     0.558472 &  0.261621 &   0.264943 &  0.692280 \\
         & occur & (1, 1) &  0.652593 &  0.323639 &     0.603434 &  0.312905 &   0.447263 &  0.683130 \\
         &       & (1, 2) &  0.672225 &  0.307046 &     0.603767 &  0.305911 &   0.453852 &  0.701811 \\
         &       & (1, 3) &  0.678005 &  0.294448 &     0.599772 &  0.299541 &   0.441820 &  0.707983 \\
newton\_ME & tfidf & (1, 1) &  0.682614 &  0.250871 &     0.587014 &  0.276658 &   0.319947 &  0.706660 \\
         &       & (1, 2) &  0.684636 &  0.230160 &     0